In [1]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import engine
import pickle
import numpy as np

In [2]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_features(trackID):
        return sp.audio_features(trackID)[0]

def get_markets(trackID):
    foo = sp.track(trackID)['available_markets']
    if foo != []:
        return 1
    else:
        return 0

In [ ]:
hashed_df = pickle.load( open( "./song_list_v4_hashed.pkl", "rb"))

In [5]:
con = sqlite3.connect("song_list11.db")
df = pd.read_sql_query("SELECT * from songs", con)
con.close()

In [10]:
df = df.drop(['level_0'], axis=1)

In [11]:
df.head(1)

,index,songid,popularity,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,...,loudness,mode,speechiness,tempo,time_signature,track,valence,genres,genres_stripped,availability
0,0,5PS5dpaLogPzYU9hWiWyZb,0,0.0013,Karkkiautomaatti,0.487,157307.0,0.678,0.0551,9.0,...,-7.78,1.0,0.0495,149.94,4.0,Tanssi vaan,0.87,"['finnish indie', 'suomi rock']","'finnish indie', 'suomi rock'",1.0


In [13]:
engine = create_engine('sqlite:///song_list1.db', echo=False)
df.to_sql('songs', con=engine)
engine.execute('SELECT * FROM songs;').fetchone()

(0, 0, '5PS5dpaLogPzYU9hWiWyZb', '0', 0.0013, 'Karkkiautomaatti', 0.487, 157307.0, 0.678, 0.0551, 9.0, 0.0846, -7.78, 1.0, 0.0495, 149.94, 4.0, 'Tanssi vaan', 0.87, "['finnish indie', 'suomi rock']", "'finnish indie', 'suomi rock'", 1.0)

In [14]:
for row_index in hashed_df['songid'][365013:574019]:
    songid = row_index
    #print(songid)
    markets = get_markets(songid)
    #print(markets)
    try:
        engine.execute(f'UPDATE songs SET availability = "{markets}" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print(f"couldn't insert {markets} into table")
        continue

couldn't insert 1 into table
couldn't insert 1 into table
couldn't insert 1 into table
retrying ...1secs
retrying ...1secs


ConnectionError: ('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')"))